In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm

In [2]:
graph = "stexpanded"
embeddings = "dogtag_bgelarge"

In [3]:
embeddings_file = "./_input/node_embeddings/" + embeddings + "/" + graph + ".json"
graph_file = "_input/triples/" + graph + ".triples"
neighborhood_embeddings_file = "./_input/neighborhood_embeddings/" + embeddings + "/" + graph + ".json"

In [ ]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [4]:
graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_nx = nx.Graph()
for row in graph_df.itertuples():
    G_nx.add_edge(int(row[1]), int(row[2]))
print(G_nx)

Graph with 34314 nodes and 254899 edges


In [ ]:
graph_file = "_input/filtered_triples/" + graph + ".triples"
graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_nx = nx.Graph()
for row in graph_df.itertuples():
    G_nx.add_edge(int(row[1]), int(row[2]))
print(G_nx)

In [ ]:
embeddings_dict = dict()

for n in tqdm(G_nx.nodes):
    
    neighbors = nx.neighbors(G_nx, int(n))
    embeddings = []

    try:
        selected_embedding = df_T.loc[int(n)]
        embeddings.append(selected_embedding)
    except KeyError:
        print(str(int(n)) + " --- Key not found anywhere!")
        pass

    for node in neighbors:
        try:
            selected_embedding = df_T.loc[node]
            embeddings.append(selected_embedding)
        except KeyError:
            print(str(node) + " --- Key not found anywhere!")
            pass

    neighborhood_embedding = np.average(embeddings, axis=0)
    embeddings_dict.update({n: list(neighborhood_embedding)})


with open(neighborhood_embeddings_file, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)